In [1]:
from ztf_hdf5 import Write_LightCurve, Read_LightCurve, Plot_LightCurve
import csv
from astropy.table import Table
import os
import numpy as np
import operator
from astropy.table import QTable, Table, Column, vstack
import astropy

In [3]:
# Read meta data file
inputDir = 'dataLC'
class_meta = Read_LightCurve(file_name='meta_simu_nb.hdf5',inputDir=inputDir)
read_meta = class_meta.get_table(path='meta')

#read_meta

In [6]:
# Read the csv file with your selection

current_dir = os.environ.get('PWD')
print('current dir',current_dir)
csv_dir = '{}/ztf_stage/csv'.format(current_dir)
selec_file = os.path.join(csv_dir, 'selection_tab.csv')
selec_seuil = os.path.join(csv_dir, 'seuil_name_selec.csv')

table = Table.read(selec_file) #Lecture du fichier csv
table_2 = Table.read(selec_seuil) #Lecture du fichier csv

print(table,'\n\n', table_2)

current dir /home/philippe/ZTF/simu
 col  lim_col type     op1    
----- ------- ---- -----------
phase       0  int operator.le
phase       0  int operator.ge
phase     -10  int operator.le
phase      20  int operator.ge
 band    ztfr  str operator.eq
 band    ztfg  str operator.eq
 band    ztfi  str operator.eq 

 seuil  name_selec 
----- ------------
    4   nb_phase<0
   10   nb_phase>0
    1 nb_phase<-10
    1  nb_phase>20
    0 nb_band_ztfr
    0 nb_band_ztfg
    0 nb_band_ztfi


# Adding len of different table in function of the different selection on the table of meta data

In [8]:
import operator
def selec_phase(Tab, csvFile, limSNR, selec=False, Verbose=False, Plot=False):
    
    Tab['SNR'] = Tab['flux'] / Tab['fluxerr']
    Tab['phase'] = (Tab['time'] - Tab.meta['t0']) / (1-Tab.meta['z'])
    mask_SNR = Tab['SNR'] >= limSNR
    Tab = Tab[mask_SNR]
    len_tab = []
    
    for i, cols in enumerate(csvFile):
        if cols['col'] not in Tab.colnames:
            print(cols['col'], 'This column doesnt exist') 
    
        else:
            col = cols['col']
            op = eval(cols['op1'])
            type_ = eval(cols['type'])
            lim = type_(cols['lim_col'])
    
            mask = op(Tab[col], lim)
            new_Tab = Tab[mask]
        
            len_tab.append(len(new_Tab))
            
    if Verbose:
        print('min :', np.min(Tab['phase']), 'max :', np.max(Tab['phase']))
    if Plot:
        x = Tab['phase']
        y = Tab['flux']

        fig, axs = plt.subplots(1, 1, figsize=(10,8))
        axs.plot(x,y,marker = '*')
            
    if selec:
        return Tab
    else:
        return len_tab

In [11]:
clas2 = Read_LightCurve(file_name='data_simu_nb.hdf5')
read_data = clas2.get_table(path='SN_0')

rtot = []
r = []
rb = table_2['name_selec']
for i, row in enumerate(read_meta):
    if row['path'] == 'SN_{}'.format(i):
        read_data = clas2.get_table(path='SN_{}'.format(i))
        len_Tab = selec_phase(read_data, table, 5)
        r.append(len_Tab)
    else : 
        b = np.full((len(table_2)), -1)
        r.append(b.tolist())

rtot = np.array(r)

for i in range(0, len(rtot[0])):
    line = [row[i] for row in rtot]
    c = Column(line)
    read_meta.add_column(c, name=rb[i])
    read_meta = read_meta
    
read_meta

z,t0,x0,x1,c,mwebv,ra,dec,mwebv_sfd98,idx_orig,path,nb_phase<0,nb_phase>0,nb_phase<-10,nb_phase>20,nb_band_ztfr,nb_band_ztfg,nb_band_ztfi
float64,float64,float64,float64,float64,float64,float64,float64,float64,int64,str4,int64,int64,int64,int64,int64,int64,int64
0.09746273115384423,58257.91621139255,0.00031819893673549164,0.6980791898299368,0.06635615610859492,1.7180447348339056,354.86086116715535,67.22534722376244,1.7236581219059706,0,SN_0,0,0,0,0,0,0,0
0.0954508501914799,58371.81785054953,0.00027713578912904685,-0.62548765619564,0.08991019018243052,0.144720279260781,352.06966274388003,44.524529678477535,0.14370772544891622,1,SN_1,14,29,2,1,23,20,0
0.08329532838343195,58274.43713241271,0.0005157470343335358,-0.7128379150987296,-0.03658009422811561,0.05471561338037373,47.24181720812578,-12.676022518562247,0.06281925529252325,2,SN_2,0,0,0,0,0,0,0
0.05239567866061053,58231.2012471626,0.0011922257996227882,-2.6676509956069445,-0.03448578174354965,0.2748790046966648,242.8963714073767,-5.72510622133645,0.27466216424998785,5,SN_3,3,11,1,4,8,6,0
0.09496615232554739,58233.14175236984,0.00032508682678714015,-0.3301954938432617,0.09642430219244609,0.021728287763163826,123.94995776116892,2.812908049012056,0.024360625856913364,6,SN_4,3,3,1,0,3,3,0
0.07527202487012849,58421.53353253866,0.0005445907177406092,-0.2915194225651692,0.09710010223419693,0.028955382485282188,122.88362397997601,72.3087518839699,0.021226937444662287,7,SN_5,0,144,0,64,93,51,0
0.07864194673814619,58210.72134081617,0.000843278667355219,-0.1441809256096224,-0.07964318534293403,0.028054051979465203,175.80441899319186,2.7920990514452346,0.017430438951182806,8,SN_6,6,11,0,6,8,9,0
0.07636610804256264,58273.41723662158,0.0004897470307437442,-0.9579709291843637,0.07850842542752962,0.027115198698569112,190.470983126126,13.051813235586533,0.023295795365418693,9,SN_7,14,48,2,9,30,24,8


# We apply our selection criteria

In [7]:
def selec(list_lenTable, csvFile):
    
    res = 1
    len_table = []
    
    for i, len_selec in enumerate(list_lenTable):
        if res:
            res &= operator.ge(len_selec, csvFile['seuil'][i])
        if not res:
            break
            
    return res

In [8]:
R, sel_list = [], []

for i, path in enumerate(read_meta['path']):
    class_data = Read_LightCurve(file_name='data_simu_nb.hdf5')
    if path == 'SN_{}'.format(i):
        lc = class_data.Read_file(path=path)
        lc_len = selec_phase(lc, table, 5, selec=False)
        sel = selec(lc_len, table_2)
        R.append(sel)
    else:
        R.append(0)

pass_selec = Column(R)

if 'pass selec' in read_meta.colnames:
    read_meta['pass selec'] = pass_selec
else :
    read_meta.add_column(pass_selec, name = 'pass selec')
    
read_meta

/var/folders/8n/j9hb298x4xq39l_vtv55kkp00000gp/T/ipykernel_86560/1317654429.py:4: RuntimeWarning: invalid value encountered in true_divide
  SNR = Tab['flux'] / Tab['fluxerr']


z,t0,x0,x1,c,mwebv,ra,dec,mwebv_sfd98,idx_orig,path,nb_phase<0,nb_phase>0,nb_phase<-10,nb_phase>20,nb_band_ztfr,nb_band_ztfg,nb_band_ztfi,pass selec
float64,float64,float64,float64,float64,float64,float64,float64,float64,int64,str7,int64,int64,int64,int64,int64,int64,int64,int64
0.09665627314283415,58168.04293368102,0.000778549045538808,0.8465494489108286,-0.12065071579152142,0.5315498204899775,328.40296803240403,63.641328219468534,0.5316688736326702,3,SN_0,0,0,0,0,0,0,0,0
0.06768985090609805,58249.37422072563,0.0005307166801524733,-0.985044384308288,0.09295633259478575,0.03949753734614846,151.59797399285733,23.25922126621252,0.029108997852342226,4,SN_1,4,8,0,2,8,4,0,0
0.052395678566137256,58236.173004001204,0.0017161841952440483,0.8996502616486605,-0.06269334858797164,0.04234273972677904,256.8127136612033,31.2443105117884,0.032614192437054663,5,SN_2,0,11,0,9,8,3,0,0
0.09496615230696706,58311.20197489228,0.0006119488225370554,0.7980226932864936,-0.050078362872994435,0.012269386001239686,220.163406234109,56.56643570276057,0.01422503874472932,6,SN_3,42,73,21,44,68,38,9,1
0.07527202479792483,58421.07953363119,0.0007035855359408554,-1.5105624950698893,-0.07095461001228488,0.02484529150395831,143.2837834146487,31.50144070656685,0.015780586499726648,7,SN_4,0,8,0,2,4,4,0,0
0.07864194667329098,58334.909398505326,0.0009331921528894242,0.4480246584401117,-0.12112950761147448,0.055517227748341544,261.1530093052832,20.720253108136966,0.05042068232864323,8,SN_5,8,32,2,18,19,12,9,1
0.07636610797218073,58346.25393644017,0.000675087201252056,1.2499701414605355,0.04374423163876206,0.09964402215482689,103.09998252529304,41.567201172508334,0.1033626321938897,9,SN_6,1,81,0,59,49,33,0,0
0.0708829178829646,58482.67225698298,0.0010685032540131056,0.27341357632480395,-0.0965271411021003,0.9438404584052268,287.12016978557534,2.3269498938275,0.9433175536280446,10,SN_7,0,0,0,0,0,0,0,0
0.06350687289352495,58285.42305707414,0.0010296304646536993,-0.4535401128481856,-0.023769855002025764,0.010812825738655957,232.86450937143496,48.44589718330789,0.01271561381923838,11,SN_8,86,188,24,94,156,108,10,1


In [9]:
if os.path.isfile('dataLC/meta_selected.hdf5'):
    print('File already exist : file remove')
    os.remove('dataLC/meta_selected.hdf5')
else:
    print('')
    
astropy.io.misc.hdf5.write_table_hdf5(read_meta, 'dataLC/meta_selected.hdf5', path='meta', append=True,
                                                  overwrite=True, serialize_meta=False)

File already exist : file remove


In [10]:
class_meta = Read_LightCurve(file_name='meta_selected.hdf5')
new_read_meta = class_meta.Read_file(path='meta')
new_read_meta

z,t0,x0,x1,c,mwebv,ra,dec,mwebv_sfd98,idx_orig,path,nb_phase<0,nb_phase>0,nb_phase<-10,nb_phase>20,nb_band_ztfr,nb_band_ztfg,nb_band_ztfi,pass selec
float64,float64,float64,float64,float64,float64,float64,float64,float64,int64,str7,int64,int64,int64,int64,int64,int64,int64,int64
0.09665627314283415,58168.04293368102,0.000778549045538808,0.8465494489108286,-0.12065071579152142,0.5315498204899775,328.40296803240403,63.641328219468534,0.5316688736326702,3,SN_0,0,0,0,0,0,0,0,0
0.06768985090609805,58249.37422072563,0.0005307166801524733,-0.985044384308288,0.09295633259478575,0.03949753734614846,151.59797399285733,23.25922126621252,0.029108997852342226,4,SN_1,4,8,0,2,8,4,0,0
0.052395678566137256,58236.173004001204,0.0017161841952440483,0.8996502616486605,-0.06269334858797164,0.04234273972677904,256.8127136612033,31.2443105117884,0.032614192437054663,5,SN_2,0,11,0,9,8,3,0,0
0.09496615230696706,58311.20197489228,0.0006119488225370554,0.7980226932864936,-0.050078362872994435,0.012269386001239686,220.163406234109,56.56643570276057,0.01422503874472932,6,SN_3,42,73,21,44,68,38,9,1
0.07527202479792483,58421.07953363119,0.0007035855359408554,-1.5105624950698893,-0.07095461001228488,0.02484529150395831,143.2837834146487,31.50144070656685,0.015780586499726648,7,SN_4,0,8,0,2,4,4,0,0
0.07864194667329098,58334.909398505326,0.0009331921528894242,0.4480246584401117,-0.12112950761147448,0.055517227748341544,261.1530093052832,20.720253108136966,0.05042068232864323,8,SN_5,8,32,2,18,19,12,9,1
0.07636610797218073,58346.25393644017,0.000675087201252056,1.2499701414605355,0.04374423163876206,0.09964402215482689,103.09998252529304,41.567201172508334,0.1033626321938897,9,SN_6,1,81,0,59,49,33,0,0
0.0708829178829646,58482.67225698298,0.0010685032540131056,0.27341357632480395,-0.0965271411021003,0.9438404584052268,287.12016978557534,2.3269498938275,0.9433175536280446,10,SN_7,0,0,0,0,0,0,0,0
0.06350687289352495,58285.42305707414,0.0010296304646536993,-0.4535401128481856,-0.023769855002025764,0.010812825738655957,232.86450937143496,48.44589718330789,0.01271561381923838,11,SN_8,86,188,24,94,156,108,10,1
